<h1 style='font-weight:bold; color:orange'> Load Data </h1>

In [7]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyhive import hive

In [ ]:
# fetch dataset from uci machine learning respository
adult = fetch_ucirepo(id=2)
X = adult.data.features
y = adult.data.targets
df = pd.concat([X, y], axis=1)

df.to_csv('adult.csv', index=False, header=False)

In [8]:
# establish connection hive Apache Hive
conn = hive.Connection(
    host="localhost",
    port=10000,
    username="maria_dev",
)

cursor = conn.cursor()

def query_hive(command):
    cursor.execute(command)
    for result in cursor.fetchall():
        print(result)

In [9]:
# create a database to store tables
cursor.execute("CREATE DATABASE IF NOT EXISTS project")

<h1 style='font-weight:bold; color:orange'> Data Preprocessing inside Apache Hive using HiveQL </h1>

In [ ]:
# view the first 10 rows from the dataset
query_hive("""
           SELECT *
           FROM project.adult
           LIMIT 10
           """)

(39, 'State-gov', 77516, 'Bachelors', 13, 'Never-married', 'Adm-clerical', 'Not-in-family', 'White', 'Male', 2174, 0, 40, 'United-States', '<=50K')
(50, 'Self-emp-not-inc', 83311, 'Bachelors', 13, 'Married-civ-spouse', 'Exec-managerial', 'Husband', 'White', 'Male', 0, 0, 13, 'United-States', '<=50K')
(38, 'Private', 215646, 'HS-grad', 9, 'Divorced', 'Handlers-cleaners', 'Not-in-family', 'White', 'Male', 0, 0, 40, 'United-States', '<=50K')
(53, 'Private', 234721, '11th', 7, 'Married-civ-spouse', 'Handlers-cleaners', 'Husband', 'Black', 'Male', 0, 0, 40, 'United-States', '<=50K')
(28, 'Private', 338409, 'Bachelors', 13, 'Married-civ-spouse', 'Prof-specialty', 'Wife', 'Black', 'Female', 0, 0, 40, 'Cuba', '<=50K')
(37, 'Private', 284582, 'Masters', 14, 'Married-civ-spouse', 'Exec-managerial', 'Wife', 'White', 'Female', 0, 0, 40, 'United-States', '<=50K')
(49, 'Private', 160187, '9th', 5, 'Married-spouse-absent', 'Other-service', 'Not-in-family', 'Black', 'Female', 0, 0, 16, 'Jamaica', '<=5

In [ ]:
# count the total number of rows
query_hive("""
           SELECT COUNT(*)
           FROM project.adult
           """)

(48842,)


In [6]:
# check for null values in each column
query_hive("""
           SELECT
            COUNT (*) AS total,
            COUNT(age) AS age_not_null,
            COUNT(workclass) AS workclass_not_null,
            COUNT(final_weight) AS weight_not_null
           FROM project.adult
           """)

(48842, 48842, 48842, 48842)


In [ ]:
query_hive("""
           SELECT age, COUNT(*) AS count,
           FROM project.adult,
           GROUP BY age,
           ORDER BY age
           """)

OperationalError: TExecuteStatementResp(status=TStatus(statusCode=3, infoMessages=["*org.apache.hive.service.cli.HiveSQLException:Error while compiling statement: FAILED: ParseException line 3:11 cannot recognize input near 'FROM' 'project' '.' in selection target:17:16", 'org.apache.hive.service.cli.operation.Operation:toSQLException:Operation.java:324', 'org.apache.hive.service.cli.operation.SQLOperation:prepare:SQLOperation.java:148', 'org.apache.hive.service.cli.operation.SQLOperation:runInternal:SQLOperation.java:228', 'org.apache.hive.service.cli.operation.Operation:run:Operation.java:264', 'org.apache.hive.service.cli.session.HiveSessionImpl:executeStatementInternal:HiveSessionImpl.java:479', 'org.apache.hive.service.cli.session.HiveSessionImpl:executeStatement:HiveSessionImpl.java:454', 'org.apache.hive.service.cli.CLIService:executeStatement:CLIService.java:285', 'org.apache.hive.service.cli.thrift.ThriftCLIService:ExecuteStatement:ThriftCLIService.java:510', 'org.apache.hive.

In [ ]:
query = """
SELECT * FROM project.adult
"""

df = pd.read_sql(query, conn)
df.head()